# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
import numpy as np
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
#Load the dataset:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train.shape

(50000, 32, 32, 3)

In [4]:
y_train.shape

(50000, 1)

In [5]:
y_test.shape

(10000, 1)

In [45]:
y_train_s=y_train.reshape(50000,)
y_test_s=y_test.reshape(10000,)

In [46]:
a=[0,1,2,3,4]
b=[5,6,7,8,9]

In [47]:
np_tr_a=np.where(y_train_s<=4)
np_tr_b=np.where(y_train_s>4)
np_te_a=np.where(y_test_s<=4)
np_te_b=np.where(y_test_s>4)

In [48]:
X_train_a=X_train[np_tr_a]
y_train_a=y_train[np_tr_a]

X_train_b=X_train[np_tr_b]
y_train_b=y_train[np_tr_b]

X_test_a=X_test[np_te_a]
y_test_a=y_test[np_te_a]

X_test_b=X_test[np_te_b]
y_test_b=y_test[np_te_b]

In [49]:
print(X_train_a.shape)
print(y_train_a.shape)
print(X_train_b.shape)
print(y_train_b.shape)
print(X_test_a.shape)
print(y_test_a.shape)
print(X_test_b.shape)
print(y_test_a.shape)

(25000, 32, 32, 3)
(25000, 1)
(25000, 32, 32, 3)
(25000, 1)
(5000, 32, 32, 3)
(5000, 1)
(5000, 32, 32, 3)
(5000, 1)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [50]:
y_train_a.shape

(25000, 1)

In [51]:
y_test_a=y_test_a.reshape(5000,)
y_train_a=y_train_a.reshape(25000,)

y_test_b=y_test_a.reshape(5000,)
y_train_b=y_train_a.reshape(25000,)

In [52]:
#One-hot encoding the labels
num_classes_a = 5
num_classes_b = 10
from keras.utils import np_utils
y_train_a = np_utils.to_categorical(y_train_a, num_classes_a)
y_test_a = np_utils.to_categorical(y_test_a, num_classes_a)

y_train_b = np_utils.to_categorical(y_train_b, num_classes_b)
y_test_b = np_utils.to_categorical(y_test_b, num_classes_b)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [53]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D

#Building up a Sequential model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',input_shape = X_train_a.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
#model.add(Dense(5, activation='softmax'))
#model.summary()

#model.add(Flatten())
model.add(Dense(200, activation='relu')) 
model.add(Dense(5, activation='softmax'))

In [54]:
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [55]:
X_train_scratch = X_train_a/255
X_test_scratch = X_test_a/255

In [56]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='scratchmodel.best.hdf5', 
                               verbose=1,save_best_only=True)

In [57]:
#Fitting the model on the train data and labels.
model.fit(X_train_scratch, y_train_a, batch_size=32, epochs=10, 
          verbose=1, callbacks=[checkpointer], validation_split=0.2, shuffle=True)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 39s 2ms/step - loss: 0.3932 - acc: 0.8245 - val_loss: 0.3355 - val_acc: 0.8502

Epoch 00001: val_loss improved from inf to 0.33546, saving model to scratchmodel.best.hdf5
Epoch 2/10
20000/20000 [==============================] - 20s 1ms/step - loss: 0.3266 - acc: 0.8545 - val_loss: 0.3147 - val_acc: 0.8625

Epoch 00002: val_loss improved from 0.33546 to 0.31470, saving model to scratchmodel.best.hdf5
Epoch 3/10
20000/20000 [==============================] - 20s 994us/step - loss: 0.2966 - acc: 0.8711 - val_loss: 0.2778 - val_acc: 0.8812

Epoch 00003: val_loss improved from 0.31470 to 0.27780, saving model to scratchmodel.best.hdf5
Epoch 4/10
20000/20000 [==============================] - 19s 946us/step - loss: 0.2712 - acc: 0.8831 - val_loss: 0.2664 - val_acc: 0.8850

Epoch 00004: val_loss improved from 0.27780 to 0.26643, saving model to scratchmodel.best.hdf5
Epoch 5/10
20000/20

In [59]:
#Evaluate the model on the test data
score = model.evaluate(X_test_scratch, y_test_a)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

5000/5000 [==============================] - 2s 346us/step
Accuracy on the Test Images:  0.9082399928092957


### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
__________

In [61]:
for layer in model.layers[:7]:
   layer.trainable = False
#The default is already set to True. I have mentioned it here to make things clear.

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [62]:
model2=Sequential(model.layers[:7])

In [63]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
Total para

In [64]:
from keras.layers import BatchNormalization

In [65]:
model2.add(Dense(200, activation='relu')) 
model2.add(Dropout(0.3))
model2.add(BatchNormalization()) 
model2.add(Dense(100, activation='relu')) 
model2.add(Dropout(0.3))
model2.add(Dense(10, activation='softmax'))

In [66]:

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 64)                0         
__________

In [67]:
model2.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [68]:
X_train_scratch = X_train_b/255
X_test_scratch = X_test_b/255

In [69]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='scratchmodel2.best.hdf5', 
                               verbose=1,save_best_only=True)

In [70]:
#Fitting the model on the train data and labels.
model2.fit(X_train_scratch, y_train_b, batch_size=32, epochs=10, 
          verbose=1, callbacks=[checkpointer], validation_split=0.2, shuffle=True)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 14s 697us/step - loss: 0.2701 - acc: 0.8982 - val_loss: 0.2518 - val_acc: 0.9000

Epoch 00001: val_loss improved from inf to 0.25185, saving model to scratchmodel2.best.hdf5
Epoch 2/10
20000/20000 [==============================] - 8s 420us/step - loss: 0.2531 - acc: 0.9000 - val_loss: 0.2505 - val_acc: 0.9000

Epoch 00002: val_loss improved from 0.25185 to 0.25049, saving model to scratchmodel2.best.hdf5
Epoch 3/10
20000/20000 [==============================] - 9s 426us/step - loss: 0.2513 - acc: 0.9000 - val_loss: 0.2504 - val_acc: 0.9000

Epoch 00003: val_loss improved from 0.25049 to 0.25045, saving model to scratchmodel2.best.hdf5
Epoch 4/10
20000/20000 [==============================] - 8s 421us/step - loss: 0.2508 - acc: 0.9000 - val_loss: 0.2504 - val_acc: 0.9000

Epoch 00004: val_loss improved from 0.25045 to 0.25044, saving model to scratchmodel2.best.hdf5
Epoch 5/10
200

In [71]:
#Evaluate the model on the test data
score = model2.evaluate(X_test_scratch, y_test_b)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

5000/5000 [==============================] - 2s 314us/step
Accuracy on the Test Images:  0.8999999761581421


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [115]:
from sklearn.datasets import fetch_20newsgroups

In [116]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [117]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [118]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [119]:
twenty_train_ar=twenty_train.data

In [120]:
twenty_train_ar=np.array(twenty_train_ar)

In [121]:
twenty_train.target.shape

(2257,)

In [122]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [123]:
#twenty_train

In [124]:
twenty_test_ar=np.array(twenty_test.data)

In [125]:
twenty_test_ar.shape

(1502,)

In [94]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [126]:
import pandas as pd
import numpy as np
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import unicodedata
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kalya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [127]:
tokenizer = ToktokTokenizer()
stopword_list = stopwords.words('english')
#stopword_list.remove('no')
#stopword_list.remove('not')
nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)

In [128]:
def preprocess(text, is_lower_case=False, remove_digits=False):
    try:
        #remove_stopwords
        tokens = tokenizer.tokenize(text)
        tokens = [token.strip() for token in tokens]
        if is_lower_case:
            filtered_tokens = [token for token in tokens if token not in stopword_list]
        else:
            filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
        filtered_text = ' '.join(filtered_tokens)
        
        #remove_special_characters
        pattern = r'[^a-zA-Z\s]' if not remove_digits else r'[^a-zA-Z\s]'
        text = re.sub(pattern, '', filtered_text)
        
        #lemmatize_text
        text = nlp(text)
        text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
        
        #strip_html_tags
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text()
        
        #remove_accented_chars
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return text
        
    except Exception as e:
        return ""

In [129]:
twenty_train_ar = [preprocess(data) for data in twenty_train_ar]

In [130]:
twenty_train_ar=np.array(twenty_train_ar)

In [131]:
twenty_train_ar[0]

'  sdcityacuk   Michael Collier   Subject   Converting image HP LaserJet III   NntpPostingHost   hampton Organization   City University line    anyone know good way   standard pc applicationPD utility   convert tifimgtga file LaserJet iii format would also like   convert HPGL   HP plotter   file please email response correct group   thank advance Michael   Michael Collier   Programmer   Computer Unit   email   MPCollierukaccity City University   Tel     x London   Fax     ECV HB'

In [132]:
twenty_test_ar = [preprocess(data) for data in twenty_test_ar]

In [133]:
twenty_test_ar=np.array(twenty_test_ar)

In [154]:
from sklearn.feature_extraction.text import TfidfVectorizer
# list of text documents
text = twenty_train_ar
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
train_data_features=vectorizer.fit(text)
#train_data_features
vect_data= vectorizer.transform(text)
x_train_df=pd.DataFrame(vect_data.todense(),columns=vectorizer.get_feature_names())


In [155]:
x_train_df.shape

(2257, 30375)

In [156]:
x_test_df=vectorizer.transform(twenty_test_ar)

In [157]:
x_test_df.shape

(1502, 30375)

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [111]:
from sklearn.linear_model import LogisticRegression as lr

In [158]:
lr_model = lr()
lr_model.fit(x_train_df, twenty_train.target)

C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\kalya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [159]:
lr_model.score(x_train_df , twenty_train.target)  

0.9898094816127603

Train Accuracy = 98.98%

In [162]:
lr_model.score(x_test_df,twenty_test.target)

0.8988015978695073

Test Accuracy = 89.88%

In [160]:
y_pred_lr=lr_model.predict(x_test_df)

In [150]:
from sklearn import metrics

In [151]:
print(metrics.classification_report(twenty_test.target, y_pred_lr))
print(metrics.confusion_matrix(twenty_test.target, y_pred_lr))

              precision    recall  f1-score   support

           0       0.92      0.73      0.82       319
           1       0.88      0.98      0.93       389
           2       0.93      0.92      0.92       396
           3       0.87      0.94      0.91       398

   micro avg       0.90      0.90      0.90      1502
   macro avg       0.90      0.89      0.89      1502
weighted avg       0.90      0.90      0.90      1502

[[233  15  18  53]
 [  3 380   6   0]
 [  5  27 363   1]
 [ 11  10   3 374]]
